Retrieve models from BioModels.
Adapted from

In [37]:
import os
import subprocess
import sys
import zipfile as z
import requests

try:
    from bioservices import BioModels
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "bioservices"])
    from bioservices import BioModels

# Set custom directory for storing the models (outside of current directory)
modelsDir = '../models'  # Change this path as needed

# Create models directory if it doesn't exist
if not os.path.exists(modelsDir):
    os.makedirs(modelsDir)

s = BioModels()

def get_filtered_models() -> list:
    query_parts_dc = {
        'mode': '*:*',
        'species': 'TAXONOMY:9606',
        'curation_status': 'curationstatus:"Manually curated"',
        'formats': 'modelformat:"SBML"',
        'kw': 'submitter_keywords:"Immuno-oncology"'
    }

    query_parts = [value for value in query_parts_dc.values() if value]
    query = " AND ".join(query_parts)

    print(f"Generated query: {query}")

    try:
        # Perform the search with the generated query
        search_results = s.search(query)

        if not search_results or 'models' not in search_results or not search_results['models']:
            print("No models found.")
            return []

        models_and_metadata = []
        for model in search_results['models']:
            model_id = model['id']
            metadata = {
                'name': model['name'],
                'organism': model.get('organism', ''),
                'curation_status': model.get('curationstatus', ''),
                'go_terms': model.get('go', [])
            }
            models_and_metadata.append({'model_id': model_id, 'metadata': metadata})

        return models_and_metadata

    except Exception as e:
        print(f"Error during search: {e}")
        return []

def download_biomodels(directory: str, num_per_download=100):
    """downloads sbml models from the curated section of biomodels

    Biomodels has a strict limit of 100 models per download so in this
    function we have to dance around this fact and compute appropriate
    chunk sizes.

    :param directory: (str) where to put zip files containing num_per_download sbml models
    :param num_per_download: How many sbml models to download at a time.
    :return:
    """
    if num_per_download > 100:
        raise ValueError("Maximum number of models that can be downloaded at a time is 100")

    # do index math.
    total_models = get_number_of_curated_models()
    num_downloads = int(total_models / num_per_download)
    remainder = total_models % num_per_download
    if remainder > 0:
        num_downloads += 1

    filenames = []

    start = 1
    for download_number in range(1, num_downloads + 1):
        if download_number == num_downloads:
            # handle last, which may have remainder
            end = total_models + 1  # account for 0 indexed python, 1 indexed biomodels
        else:
            end = (download_number * num_per_download) + 1  # account for 0 indexed python, 1 indexed biomodels
        # do something ...

        fname = os.path.join(directory, f"Biomodels{start}-{end - 1}.zip")
        filenames.append(fname)

        if os.path.isfile(fname):
            os.remove(fname)

        biomodels_ids = [f"BIOMD{i:010}" for i in range(start, end)]

        s.search_download(biomodels_ids, output_filename=fname)
        print(f"Biomodels models from id {start} to {end - 1} saved to {fname}")

        start = end

    # consolidate zips
    with z.ZipFile(filenames[0], 'a') as z1:
        for fname in filenames[1:]:
            zf = z.ZipFile(fname, 'r')
            for n in zf.namelist():
                z1.writestr(n, zf.open(n).read())

    # rename first zip
    biomodels_zip = os.path.join(directory, "biomodels.zip")
    if not os.path.isfile(biomodels_zip):
        os.rename(filenames[0], biomodels_zip)

    # try to get rid of the rest. Windows likes to hang on to them though so might fail
    for i in range(1, len(filenames)):
        try:
            os.remove(filenames[i])
        except Exception:
            continue

    return filenames

if __name__ == "__main__":
    # Set to True to actually perform the download
    ACTIVATE_DOWNLOAD = True

    if ACTIVATE_DOWNLOAD:
        models_and_metadata = get_filtered_models()
        if models_and_metadata:
            final_zip = download_biomodels(modelsDir, models_and_metadata)
            print(f"All models saved in {final_zip}")


WARNING [bioservices.BioModels:130]:  The URL (https://www.ebi.ac.uk/biomodels) provided cannot be reached.
INFO    [bioservices.BioModels:363]:  Initialising BioModels service (REST)


Generated query: *:* AND TAXONOMY:9606 AND curationstatus:"Manually curated" AND modelformat:"SBML" AND submitter_keywords:"Immuno-oncology"


INFO    [bioservices.BioModels:171]:  Saving file BIOMD0000000919.zip


Error on model BIOMD0000000919: 'NoneType' object has no attribute 'encode'


INFO    [bioservices.BioModels:171]:  Saving file BIOMD0000000921.zip


...

Error on model BIOMD0000001023: 'NoneType' object has no attribute 'encode'


INFO    [bioservices.BioModels:171]:  Saving file BIOMD0000001024.zip


Error on model BIOMD0000001024: 'NoneType' object has no attribute 'encode'


INFO    [bioservices.BioModels:171]:  Saving file BIOMD0000001025.zip


Error on model BIOMD0000001025: 'NoneType' object has no attribute 'encode'
All models saved in ../models/biomodels.zip
